## LAB 5 Jak działa GPT
### Tokenizacja z wykorzystaniem algorytmu BPE

Celem laboratorium jest:

* implementacja algorytmu BPE
* zrozumienie koncepcji tokenizacji

#### Źródła:
* https://github.com/karpathy/minbpe
* https://youtu.be/zduSFxRajkE?si=pHViKDX4I6yrCqf6

Imię i nazwisko: .....

Punktacja:
* Prawidłowa implementacja 6 pkt.
* Wnioski 2 pkt.

In [ ]:
from collections import Counter
import requests

In [ ]:
# jako tekst ponownie wykorzystamy HPMOR
# Eliezer Yudkowsky, Harry Potter and the Methods of Rationality https://hpmor.com/
url = "https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/data/hpmor_part.txt?raw=true"
response = requests.get(url)
text = response.text

In [ ]:
text[:100]

In [ ]:
# zamieniamy tekst na bajty z kodowania utf8
text_bytes = list(text.encode('utf8'))
print(text_bytes[:50])

In [ ]:
# tworzymy początkowy słownik - ponieważ pracujemy na bajtach to mają one wartości od 0 do 255
vocab = {i: bytes([i]) for i in range(256)}

In [ ]:
print(vocab)

#### Algorytm BPE
Będziemy implementować kroki algorytmu BPE:
1. znaleźć najczęstszą parę tokenów
2. zastąpić ją nowym tokenem
3. powtórzyć kroki 1 i 2, aż do osiągnięcia zadanego limitu liczby połączeń (n_merges)

##### Zaimplementuj funkcję znajdującą najczęstszą parę tokenów
1. stwórz listę **par kolejnych** tokenów
2. zlicz wystąpienia każdej pary (wskazówka: Counter)
3. znajdź parę z największą liczbą wystąpień (wskazówka: most_common i końcówka notebooka z wykładu)
4. zwróć parę


In [ ]:
def find_most_frequent_pair(tokens_list):
    #TODO
    #TODO
    #TODO

In [ ]:
# na początku nasza lista tokenów to lista bajtów z tekstu
tokens_list = list(text_bytes)

In [ ]:
most_freq_pair = find_most_frequent_pair(tokens_list)
most_freq_pair

In [ ]:
assert most_freq_pair == (101, 32)

In [ ]:
# znaki tworzące najczęstszą parę tokenów
print(vocab[most_freq_pair[0]])
print(vocab[most_freq_pair[1]])

#### Napisz funkcję, która zastępuje wybraną parę tokenów nowym tokenem
*Uwaga: to jest miejsce gdzie jest najwięcej pracy i najłatwiej się pomylić, dalej będzie łatwiej*
* uwaga: jeśli zastępujemy to trzeba w przetwarzaniu ominąć następny token
    * tzn. w poniższym przykładzie zastępując (1,2) przez 5, to przetwarzanie 2 pomijamy
* uwaga: proszę nie gubić ostatniego tokenu

Przykład: replace([1,2,3,4], (1,2), 5) -> [5,3,4]

Kroki:
1. stwórz nową listę tokenów
2. przejdź pętlą for przez listę tokenów
    * sprawdzaj parę token i następny token
    * jeśli napotkasz wybraną parę tokenów, to dodaj nowy token do nowej listy i pomiń następny token
    * jeśli nie napotkasz pary tokenów, to dodaj aktualny token do nowej listy
3. jeśli nie został pominięty ostatni token, to dodaj go do nowej listy
4. zwróć nową listę tokenów


In [ ]:
def replace(tokens_list, pair, new_token):
    new_tokens_list = []
    # TODO
    return new_tokens_list 

In [ ]:
assert replace([1, 2, 3, 4], (1,2), 5) == [5, 3, 4]

In [ ]:
next_token_idx = 256 # pierwszy wolny indeks
new_tokens_list = replace(tokens_list, most_freq_pair, next_token_idx)

In [ ]:
# tokeny przed zamianą (20 pierwszych)
print(tokens_list[:20])

In [ ]:
# tokeny po połączeniu (20 pierwszych)
print(new_tokens_list[:20])

#### Zaimplementujemy 50 połączeń

Potrzebujemy słownika merges, który będzie przechowywał mapowanie par tokenów na nowe tokeny.
Kluczami są pary tokenów, a wartością nowy token.
Przykład:
{(101, 32): 256,
 (32, 116): 257,
 (116, 32): 258}

 Słownik ten będzie nam potrzebny do kodowania i dekodowania tekstu.

 Kroki:
1. stwórz pusty słownik merges
2. przejdź pętlą while, aż do osiągnięcia zadanego limitu liczby połączeń (n_merges) (warto sprawdzać czy lista tokenów ma więcej niż 1 token)
    * znajdź najczęstszą parę tokenów
    * zastąp ją nowym tokenem
    * dodaj parę do słownika merges
    * zwiększ indeks nowego tokenu
    * zwiększ licznik połączeń

In [ ]:
n_merges = 50
next_token_idx = 256
tokens_list = list(text_bytes)
merges = {}
merges_counter = 0
while merges_counter < n_merges and len(tokens_list) > 1:
    most_freq_pair = # TODO
    tokens_list =  # TODO
    merges[most_freq_pair] =  # TODO
    # TODO
    # TODO

In [ ]:
assert len(tokens_list) == 10601

In [ ]:
print(tokens_list[:100])

In [ ]:
merges

In [ ]:
# aktualizaujemy nasz słownik o nowe tokeny
for (t1, t2), tm in merges.items():
    vocab[tm] = vocab[t1] + vocab[t2]

In [ ]:
# przejrzyj tokeny powyżej 256 - jakie są?
print(vocab)

In [ ]:
# zamiana tekstu na tokeny
def encode(text):
    text_bytes = list(text.encode('utf8'))
    tokens_list = list(text_bytes)
    for pair, token in merges.items():
        tokens_list = replace(tokens_list, pair, token)
    return tokens_list

In [ ]:
# zamiana tokenów na tekst
def decode(tokens_list):
    text_bytes = [vocab[t] for t in tokens_list]
    merged_bytes = b''.join(text_bytes)
    return merged_bytes.decode('utf8')

In [ ]:
tokens_list = encode(text)
print(tokens_list[:100])

In [ ]:
# powinniśmy otrzymać z powrotem tekst
decode(tokens_list)

In [ ]:
print('długość tekstu:', len(text))
print('długość tekstu w bajtach:', len(list(text.encode('utf8'))))
print('liczba tokenów po tokenizacji:', len(tokens_list))

### Klasa tokenizer
* korzystając z kodu powyżej zaimplementuj klasę Tokenizer
* zaimplementuj funkcje:
    * train
    * encode
    * decode
    * find_most_frequent_pair
    * replace

#### Uwaga
* Przed testmi implementacji warto zrestartować kernel, żeby być pewnym, że nie korzysta się ze starych zmiennych globalnych
* Potem trzeba ponownie wywołać komórki z importem bibliotek i wczytaniem tekstu (dwie pierwsze komórki z kodem od góry)

In [ ]:
class Tokenizer:
    def __init__(self):
        self.vocab = {} # słownik tokenów - numer tokenu -> tekst tokenu
        self.merges = {} # słownik połączeń - para tokenów -> nowy token
        self.next_token_idx = 0 # indeks nowego tokenu (pierwszy wolny indeks)

    def train(self, text, n_merges=100): # n_merges to liczba połączeń
        self.vocab = {i: bytes([i]) for i in range(256)}
        self.next_token_idx = 256
        tokens_list = list(text.encode('utf8'))
        self.merges = {}
        # wykonaj zadaną liczbę połączeń, aktualizując listę tokenów, słownik merges i indeks nowego tokenu
        #TODO

        # aktualizacja słownika tokenów
        for (t1, t2), tm in self.merges.items():
            self.vocab[tm] = self.vocab[t1] + self.vocab[t2]

    def encode(self, text):
        # zamiana tekstu na listę tokenów
        # TODO
        return tokens_list

    def decode(self, tokens_list):
        # zamiana listy tokenów na tekst
        # TODO
        return #TODO

    def find_most_frequent_pair(self, tokens_list):
        # znajdź najczęstszą parę tokenów
        # TODO
        return #TODO

    def replace(self, tokens_list, pair, new_token):
        # zamiana pary tokenów na nowy token
        # funkcja zwraca nową listę tokenów
        new_tokens_list = []
        # TODO
        return new_tokens_list 

In [ ]:
# tworzymy tokenizer
tokenizer = Tokenizer()

In [ ]:
# uczymy tokenizer
tokenizer.train(text, n_merges=50)

In [ ]:
print(tokenizer.merges)

In [ ]:
print(tokenizer.vocab)

In [ ]:
# sprawdzamy czy zamiana działa
tokenizer.replace([1,2,3,4], (1,2), 5)

In [ ]:
# tokenizujemy fragment tekstu
tokens_list = tokenizer.encode(text[:70])
print(tokens_list)

In [ ]:
text[:70]

In [ ]:
# i dekodujemy fragment tekstu (powinniśmy dostać z powrotem ten sam tekst)
tokenizer.decode(tokens_list)

In [ ]:
print(len(text))
print(len(tokenizer.encode(text)))

#### Uczenie na dłuższym korpusie tekstu po polsku i po angielsku
* Wykorzystamy polski przekład Tajemnicy Baskervilleow
    * Tekst pobrany z https://www.gutenberg.org/
* Oraz dłuższy fragment (10 rozdziałów) HPMOR

In [ ]:
url = "https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/data/hpmor_chapters_1-10.txt?raw=true"
response = requests.get(url)
text_eng = response.text
url = "https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/data/TajemnicaBaskervilleow.txt?raw=true"
response = requests.get(url)
text_pl = response.text

In [ ]:
print(text_pl[:1000])

In [ ]:
print(text_eng[:1000])

In [ ]:
print('długość polskiego tekstu:', len(text_pl))
print('długość angielskiego tekstu:', len(text_eng))

In [ ]:
tokenizer_pl = Tokenizer()
tokenizer_pl.train(text_pl)

In [ ]:
for k, v in tokenizer_pl.vocab.items():
    if k > 256:
        print(k, v.decode('utf8'))

In [ ]:
tokens = tokenizer_pl.encode(text_pl)
print('długość polskiego tekstu:', len(text_pl))
print('długość polskiego tekstu w bajtach:', len(list(text_pl.encode('utf8'))))
print('liczba tokenów po tokenizacji:', len(tokens))

In [ ]:
assert len(tokens)==148913

#### Powtórz powyższe dla tekstu angielskiego
* utwórz obiekt tokenizer_eng
* naucz tokenizer_eng na tekście angielskim
* wypisz słownik tokenów
* tokenizuj tekst angielski
* wypisz długość tekstu w bajtach i liczbę tokenów


In [ ]:
# TODO: Utworzyć obiekt tokenizer_eng i nauczyć go na tekście angielskim

In [ ]:
# TODO: Wyświetlić słownik tokenów

In [ ]:
# TODO: Tokenizować tekst angielski
# TODO: Wyświetlić długość tekstu w bajtach i liczbę tokenów

In [ ]:
assert len(tokens)==151373

#### Wykonaj tokenizację tekstu polskiego z wykorzystaniem tokenizatora nauczonego na tekście angielskim
* wypisz długość tekstu w tokenach po tokenizacji angielskiego tokenizera
* wypisz długość tekstu w tokenach po tokenizacji polskiego tokenizera


In [ ]:
# TODO: Wykonać tokenizację tekstu polskiego z wykorzystaniem tokenizatora nauczonego na tekście angielskim
# TODO: Wyświetlić długość tekstu w tokenach po tokenizacji angielskiego tokenizera
# TODO: Wyświetlić długość tekstu w tokenach po tokenizacji polskiego tokenizera


### Pytania
1. Jakie tokeny uzyskaliśmy dla uczenia na tekście angielskim? A jakie dla uczenia na tekście polskim? Czemu są różne?
2. Dla tekstu angielskiego liczba znaków i liczba bajtów jest taka sama. Dlaczego dla tekstu polskiego liczba bajtów jest większa?
3. Na końcu wykorzystujemy tokenizator nauczony na tekście angielskim do tokenizacji tekstu polskiego. Jak ocenisz wynik? Czemu liczba tokenów różni się od liczby tokenów uzyskanej przy tokenizacji z wykorzystaniem tokenizatora nauczonego na tekście polskim?